In [44]:
# def play_connect():
#     client = Minio(endpoint="minio:9000", 
#                    access_key="wFw2nWmmNHNivuqh", 
#                    secret_key="Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs", 
#                    secure=True)
#     bucket_setup(client)
#     return client

import logging

# Cria um logger com o nome 'jupyter_logger'
logger = logging.getLogger('jupyter_logger')

# Define o nível de log para DEBUG, então todas as mensagens de log serão mostradas
logger.setLevel(logging.DEBUG)

# Cria um manipulador de log que escreve as mensagens de log na saída padrão
handler = logging.StreamHandler()

# Define o nível de log do manipulador para DEBUG
handler.setLevel(logging.DEBUG)

# Cria um formatador de log que adiciona a data e hora à mensagem de log
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Adiciona o formatador ao manipulador
handler.setFormatter(formatter)

# Adiciona o manipulador ao logger
logger.addHandler(handler)

In [46]:
from minio import Minio
from minio.error import S3Error

def minion_upload(source_file,destination_file,bucket_name="raw"):
    # Create a client with the MinIO server playground, its access key
    # and secret key.
    client = Minio(endpoint="minio:9000", 
                   access_key="wFw2nWmmNHNivuqh", 
                   secret_key="Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs", 
                   secure=False)

    # The file to upload, change this path if needed
    # source_file = "raw_data.ipynb"

    # The destination bucket and filename on the MinIO server
    # bucket_name = "raw"
    # destination_file = "my-test-file.txt"

    # Make the bucket if it doesn't exist.
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
        logger.debug(f"Created bucket {bucket_name}")
    else:
        logger.debug(f"Bucket {bucket_name} already exists")

    # Upload the file, renaming it in the process
    client.fput_object(
        bucket_name, destination_file, source_file,
    )
    logger.debug(
        f"{source_file} successfully uploaded as object {destination_file} to bucket {bucket_name}"
    )

In [47]:
import json

def writing_json(data):
    data_obj = json.loads(data)
    path = f'user/{data_obj["uid"]}.json'

    # Abra o arquivo onde você deseja escrever
    with open(path, 'w') as f:
        # Use json.dump para escrever o objeto no arquivo
        json.dump(data_obj, f)
    return path

In [50]:
from confluent_kafka import Consumer
from time import sleep

class ExampleConsumer:
    broker = "kafka-broker:9092"
    topic = "user"
    group_id = "python"

    def start_listener(self):
        consumer_config = {
            'bootstrap.servers': self.broker,
            'group.id': self.group_id,
            'auto.offset.reset': 'latest',
            'enable.auto.commit': 'false',
            'max.poll.interval.ms': '86400000'
        }

        consumer = Consumer(consumer_config)
        consumer.subscribe([self.topic])
        # msg = consumer.poll(0)
        # print(msg.value())

        try:
            while True:
                #print("Listening")
                # read single message at a time
                msg = consumer.poll(0)
                
                if msg is None:
                    sleep(5)
                    continue
                if msg.error():
                    print("Error reading message : {}".format(msg.error()))
                    continue
                # You can parse message and save to data base here
                logger.debug('----------------------------')
                logger.debug("partition: {} \n offset: {} \n topic: {} \n timestamp: {} \n value: {}".format(msg.partition(), msg.offset(),msg.topic(),msg.timestamp(),msg.value()))
                #sua regra
                
                # print()
                path=writing_json(msg.value().decode("utf-8"))
                minion_upload(source_file=path,destination_file=path)
                                
                consumer.commit()

        except Exception as ex:
            print("Kafka Exception : {}", ex)

        finally:
            print("closing consumer")
            consumer.close()

In [51]:
my_consumer = ExampleConsumer()
my_consumer.start_listener()

2024-05-19 20:07:00,758 - jupyter_logger - DEBUG - ----------------------------
2024-05-19 20:07:00,758 - jupyter_logger - DEBUG - ----------------------------
2024-05-19 20:07:00,761 - jupyter_logger - DEBUG - partition: 0 
 offset: 305 
 topic: user 
 timestamp: (1, 1716149197311) 
 value: b'{"id": 8138, "uid": "ebd62abe-ba51-4489-9dd5-4eec557391c8", "password": "HsJd52Dph9", "first_name": "Warner", "last_name": "Ondricka", "username": "warner.ondricka", "email": "warner.ondricka@email.com", "avatar": "https://robohash.org/teneturdoloresnobis.png?size=300x300&set=set1", "gender": "Female", "phone_number": "+238 1-963-916-0694 x33143", "social_insurance_number": "513543843", "date_of_birth": "1977-11-17", "employment": {"title": "Customer Manufacturing Consultant", "key_skill": "Work under pressure"}, "address": {"city": "Zulaufview", "street_name": "Stephen Throughway", "street_address": "9561 Juan Hills", "zip_code": "22872", "state": "West Virginia", "country": "United States", "co

closing consumer


KeyboardInterrupt: 